# Diary Emotion Classifier



# Load in raw dataset

In [2]:
import pandas as pd

df = pd.read_csv("250_RAW_DATASET.csv")

df

,Script,Đối tượng,Vui,Buồn,Tức,Ngạc nhiên,Chán,Emotion
0,Hôm nay là một ngày tuyệt vời mà tôi không thể...,sinh viên đại học,1,0,0,0,0,Happy
1,"Hihi, hôm nay quả là một ngày thật vui. Hiện t...",Ông bố con điểm cao,1,0,0,0,0,Happy
2,Hôm nay là một ngày mà mình không bao giờ quên...,Có tiền viện phí cho Mẹ,1,0,0,0,0,Happy
3,Hôm nay là một ngày tuyệt vời mà mình sẽ không...,Tỏ tình thành công,1,0,0,0,0,Happy
4,Ngày đầu tiên tôi đưa đứa con đầu lòng đi học ...,mẹ bỉm sữa,1,0,0,0,0,Happy
...,...,...,...,...,...,...,...,...
245,"Mỗi ngày đến công trường, tôi lại cảm thấy như...",NaN,0,0,0,0,1,Bored
246,Làm việc tại quán cà phê mỗi ngày khiến tôi cả...,NaN,0,0,0,0,1,Bored
247,Làm việc trong xưởng mộc một mình mỗi ngày khi...,NaN,0,0,0,0,1,Bored
248,"Là một nhà văn, tôi dành phần lớn thời gian ở ...",NaN,0,0,0,0,1,Bored


# Word segmentation as a requirement of PhoBert model


In [3]:
script = df["Script"]
script

,Script
0,Hôm nay là một ngày tuyệt vời mà tôi không thể...
1,"Hihi, hôm nay quả là một ngày thật vui. Hiện t..."
2,Hôm nay là một ngày mà mình không bao giờ quên...
3,Hôm nay là một ngày tuyệt vời mà mình sẽ không...
4,Ngày đầu tiên tôi đưa đứa con đầu lòng đi học ...
...,...
245,"Mỗi ngày đến công trường, tôi lại cảm thấy như..."
246,Làm việc tại quán cà phê mỗi ngày khiến tôi cả...
247,Làm việc trong xưởng mộc một mình mỗi ngày khi...
248,"Là một nhà văn, tôi dành phần lớn thời gian ở ..."


In [4]:
#Import the segmentation model from HuggingFace

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [5]:
segmented_script = []

for sentence in script:

  ner_results = nlp(sentence)
  example_tok = ""
  for e in ner_results:
      if "##" in e["word"]:
          example_tok = example_tok + e["word"].replace("##","")
      elif e["entity"] =="I":
          example_tok = example_tok + "_" + e["word"]
      else:
          example_tok = example_tok + " " + e["word"]

  segmented_script.append(example_tok)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
#Check the len and type
len(segmented_script), type(segmented_script)

(250, list)

In [7]:
df["Segmented_script"] = segmented_script
df

,Script,Đối tượng,Vui,Buồn,Tức,Ngạc nhiên,Chán,Emotion,Segmented_script
0,Hôm nay là một ngày tuyệt vời mà tôi không thể...,sinh viên đại học,1,0,0,0,0,Happy,Hôm_nay là một ngày tuyệt_vời mà tôi không_th...
1,"Hihi, hôm nay quả là một ngày thật vui. Hiện t...",Ông bố con điểm cao,1,0,0,0,0,Happy,"Hihi , hôm_nay quả là một ngày thật vui . Hiệ..."
2,Hôm nay là một ngày mà mình không bao giờ quên...,Có tiền viện phí cho Mẹ,1,0,0,0,0,Happy,Hôm_nay là một ngày mà mình không bao_giờ quê...
3,Hôm nay là một ngày tuyệt vời mà mình sẽ không...,Tỏ tình thành công,1,0,0,0,0,Happy,Hôm_nay là một ngày tuyệt_vời mà mình sẽ khôn...
4,Ngày đầu tiên tôi đưa đứa con đầu lòng đi học ...,mẹ bỉm sữa,1,0,0,0,0,Happy,Ngày đầu_tiên tôi đưa đứa con đầu_lòng đi học...
...,...,...,...,...,...,...,...,...,...
245,"Mỗi ngày đến công trường, tôi lại cảm thấy như...",NaN,0,0,0,0,1,Bored,"Mỗi ngày đến công_trường , tôi lại cảm_thấy n..."
246,Làm việc tại quán cà phê mỗi ngày khiến tôi cả...,NaN,0,0,0,0,1,Bored,Làm_việc tại quán cà_phê mỗi ngày khiến tôi c...
247,Làm việc trong xưởng mộc một mình mỗi ngày khi...,NaN,0,0,0,0,1,Bored,Làm_việc trong xưởng mộc một_mình mỗi ngày kh...
248,"Là một nhà văn, tôi dành phần lớn thời gian ở ...",NaN,0,0,0,0,1,Bored,"Là một nhà_văn , tôi dành phần_lớn thời_gian ..."


# TOKENIZATION BY PHOBERT (VinAI)

In [8]:
segmented = df["Segmented_script"]
segmented

,Segmented_script
0,Hôm_nay là một ngày tuyệt_vời mà tôi không_th...
1,"Hihi , hôm_nay quả là một ngày thật vui . Hiệ..."
2,Hôm_nay là một ngày mà mình không bao_giờ quê...
3,Hôm_nay là một ngày tuyệt_vời mà mình sẽ khôn...
4,Ngày đầu_tiên tôi đưa đứa con đầu_lòng đi học...
...,...
245,"Mỗi ngày đến công_trường , tôi lại cảm_thấy n..."
246,Làm_việc tại quán cà_phê mỗi ngày khiến tôi c...
247,Làm_việc trong xưởng mộc một_mình mỗi ngày kh...
248,"Là một nhà_văn , tôi dành phần_lớn thời_gian ..."


In [9]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as IhateNumpy

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")


tokenized = []
i = 0

for script in segmented:
  print(f"Number of sentence being tokenized: {i}")
  input_ids = IhateNumpy.array(([tokenizer.encode(script)]))
  tokenized.append(input_ids)
  i+=1


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Number of sentence being tokenized: 0
Number of sentence being tokenized: 1
Number of sentence being tokenized: 2
Number of sentence being tokenized: 3
Number of sentence being tokenized: 4
Number of sentence being tokenized: 5
Number of sentence being tokenized: 6
Number of sentence being tokenized: 7
Number of sentence being tokenized: 8
Number of sentence being tokenized: 9
Number of sentence being tokenized: 10
Number of sentence being tokenized: 11
Number of sentence being tokenized: 12
Number of sentence being tokenized: 13
Number of sentence being tokenized: 14
Number of sentence being tokenized: 15
Number of sentence being tokenized: 16
Number of sentence being tokenized: 17
Number of sentence being tokenized: 18
Number of sentence being tokenized: 19
Number of sentence being tokenized: 20
Number of sentence being tokenized: 21
Number of sentence being tokenized: 22
Number of sentence being tokenized: 23
Number of sentence being tokenized: 24
Number of sentence being tokenized:

In [10]:
len(tokenized), type(tokenized[0])

(250, numpy.ndarray)

In [12]:
df["Tokenized"] = tokenized

# Padding to ensure balance data for training

In [13]:
tokenized = df["Tokenized"]
padded = []

max_len_to_padding = 700

i =0

for sentence in tokenized:
  diff = max_len_to_padding - sentence.shape[1]
  padding_tensor = IhateNumpy.zeros((diff))

  padded_sentence = IhateNumpy.hstack((sentence.squeeze(), padding_tensor))

  padded.append(padded_sentence)

In [14]:
normal_padded = []

for sample in padded:
  new_sample = sample.tolist() #Convert from numpy back to list
  normal_padded.append(new_sample)

In [15]:
df["Padded"] = padded

In [16]:
df

,Script,Đối tượng,Vui,Buồn,Tức,Ngạc nhiên,Chán,Emotion,Segmented_script,Tokenized,Padded
0,Hôm nay là một ngày tuyệt vời mà tôi không thể...,sinh viên đại học,1,0,0,0,0,Happy,Hôm_nay là một ngày tuyệt_vời mà tôi không_th...,"[[0, 3791, 8, 16, 43, 1819, 64, 70, 6021, 1323...","[0.0, 3791.0, 8.0, 16.0, 43.0, 1819.0, 64.0, 7..."
1,"Hihi, hôm nay quả là một ngày thật vui. Hiện t...",Ông bố con điểm cao,1,0,0,0,0,Happy,"Hihi , hôm_nay quả là một ngày thật vui . Hiệ...","[[0, 8043, 9599, 4, 1113, 645, 8, 16, 43, 520,...","[0.0, 8043.0, 9599.0, 4.0, 1113.0, 645.0, 8.0,..."
2,Hôm nay là một ngày mà mình không bao giờ quên...,Có tiền viện phí cho Mẹ,1,0,0,0,0,Happy,Hôm_nay là một ngày mà mình không bao_giờ quê...,"[[0, 3791, 8, 16, 43, 64, 68, 17, 789, 1323, 1...","[0.0, 3791.0, 8.0, 16.0, 43.0, 64.0, 68.0, 17...."
3,Hôm nay là một ngày tuyệt vời mà mình sẽ không...,Tỏ tình thành công,1,0,0,0,0,Happy,Hôm_nay là một ngày tuyệt_vời mà mình sẽ khôn...,"[[0, 3791, 8, 16, 43, 1819, 64, 68, 38, 17, 78...","[0.0, 3791.0, 8.0, 16.0, 43.0, 1819.0, 64.0, 6..."
4,Ngày đầu tiên tôi đưa đứa con đầu lòng đi học ...,mẹ bỉm sữa,1,0,0,0,0,Happy,Ngày đầu_tiên tôi đưa đứa con đầu_lòng đi học...,"[[0, 319, 191, 70, 111, 802, 73, 7951, 57, 222...","[0.0, 319.0, 191.0, 70.0, 111.0, 802.0, 73.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
245,"Mỗi ngày đến công trường, tôi lại cảm thấy như...",NaN,0,0,0,0,1,Bored,"Mỗi ngày đến công_trường , tôi lại cảm_thấy n...","[[0, 1106, 43, 30, 6735, 4, 70, 44, 841, 42, 6...","[0.0, 1106.0, 43.0, 30.0, 6735.0, 4.0, 70.0, 4..."
246,Làm việc tại quán cà phê mỗi ngày khiến tôi cả...,NaN,0,0,0,0,1,Bored,Làm_việc tại quán cà_phê mỗi ngày khiến tôi c...,"[[0, 11631, 35, 946, 1511, 205, 43, 122, 70, 8...","[0.0, 11631.0, 35.0, 946.0, 1511.0, 205.0, 43...."
247,Làm việc trong xưởng mộc một mình mỗi ngày khi...,NaN,0,0,0,0,1,Bored,Làm_việc trong xưởng mộc một_mình mỗi ngày kh...,"[[0, 11631, 12, 4255, 7429, 2595, 205, 43, 122...","[0.0, 11631.0, 12.0, 4255.0, 7429.0, 2595.0, 2..."
248,"Là một nhà văn, tôi dành phần lớn thời gian ở ...",NaN,0,0,0,0,1,Bored,"Là một nhà_văn , tôi dành phần_lớn thời_gian ...","[[0, 1589, 16, 2824, 4, 70, 365, 1959, 130, 25...","[0.0, 1589.0, 16.0, 2824.0, 4.0, 70.0, 365.0, ..."


# One-hot encoding emotion label for training

In [30]:
# One-hot encoding using pd.get_dummies
one_hot_encoded = pd.get_dummies(df['Emotion'], prefix='Emotion')

# Combine the one-hot encoded labels with the original DataFrame (optional)
df = pd.concat([df, one_hot_encoded], axis=1)

# Convert the DataFrame to a NumPy array
labels_array = one_hot_encoded.values

# Convert the NumPy array to a tensor
Y = torch.tensor(labels_array, dtype=torch.float32)

Y.shape, Y

(torch.Size([250, 5]),
 tensor([[0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         ...,
         [0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.]]))

# Split dataset by Sklearn function

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
# Extract the 'Padded' column as a NumPy array of objects
import numpy as np

training_feature = np.array(df['Padded'].tolist())

In [33]:
X = torch.from_numpy(training_feature).type(torch.float32)
X.shape

torch.Size([250, 700])

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [35]:
len(X_train), len(X_test), len(Y_train), len(Y_test)

(200, 50, 200, 50)

# Construct a model

In [36]:
from torch import nn

In [52]:
import torch
import torch.nn as nn

class EmotionClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(in_features=700, out_features=1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(in_features=1024, out_features=2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(in_features=2048, out_features=2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=5),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_stack(x)

model = EmotionClassification()


# Loss function and Optimizer

In [45]:
loss_function = nn.CrossEntropyLoss() #Compare logits(raw) output of model WITH REAL LABEL
optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.05)

In [39]:
#Calculate accuracy

def accuracy_fn(Y_true, Y_pred):
  correct = torch.eq(Y_true, Y_pred) #return HOW MANY CORRECT PREDICTION
  acc = (correct/len(Y_pred))*100

  return acc

# Training loop

# Using CrossEntropyLoss function

In [ ]:
#TRAINING LOOP
epochs = 1000

for epoch in range(epochs):
  model.train()

  train_logits = model(X_train)
  train_pred = torch.argmax(torch.softmax(train_logits, dim = 1), dim = 1)

  train_loss = loss_function(train_logits, Y_train)
  train_acc = accuracy_fn(Y_train, train_pred)

  optimizer.zero_grad()

  train_loss.backward()

  optimizer.step()

  #Evaluation
  model.eval()

  with torch.inference_mode():
    test_logits = model(X_test)
    test_pred = torch.argmax(torch.softmax(test_logits, dim =1), dim =1)

    test_loss = loss_function(test_logits, Y_test)
    test_acc = accuracy_fn(Y_test, test_pred)

  #Visualize
  if epoch%100 == 0:
    print(f"Epoch: {epoch} | Train loss : {train_loss} | Test loss : {test_loss}")


Epoch: 0 | Train loss : 1.7073973417282104 | Test loss : 2279571968.0
Epoch: 100 | Train loss : 1.6001217365264893 | Test loss : 1.621428370475769
Epoch: 200 | Train loss : 1.584067702293396 | Test loss : 1.6202389001846313
Epoch: 300 | Train loss : 1.6079119443893433 | Test loss : 1.6231930255889893
Epoch: 400 | Train loss : 1.6079115867614746 | Test loss : 1.6232163906097412
Epoch: 500 | Train loss : 1.6079111099243164 | Test loss : 1.6233254671096802
Epoch: 600 | Train loss : 1.6079108715057373 | Test loss : 1.6233259439468384
Epoch: 700 | Train loss : 1.6079108715057373 | Test loss : 1.6233606338500977
Epoch: 800 | Train loss : 1.6079111099243164 | Test loss : 1.6233301162719727
Epoch: 900 | Train loss : 1.6079143285751343 | Test loss : 1.6232354640960693


In [40]:
# TRAINING LOOP
epochs = 1000

# Convert one-hot encoded labels to integer labels for training and evaluation
Y_train_labels = torch.argmax(Y_train, dim=1)
Y_test_labels = torch.argmax(Y_test, dim=1)


for epoch in range(epochs):
    model.train()

    # Forward pass
    train_logits = model(X_train)

    # Get predictions using softmax and argmax
    train_pred = torch.argmax(torch.softmax(train_logits, dim=1), dim=1)

    # Calculate loss
    train_loss = loss_function(train_logits, Y_train_labels)

    # Calculate accuracy
    train_acc = accuracy_fn(Y_train_labels, train_pred)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass
    train_loss.backward()

    # Update weights
    optimizer.step()

    # Evaluation
    model.eval()

    with torch.inference_mode():
        test_logits = model(X_test)
        test_pred = torch.argmax(torch.softmax(test_logits, dim=1), dim=1)

        # Calculate test loss and accuracy
        test_loss = loss_function(test_logits, Y_test_labels)
        test_acc = accuracy_fn(Y_test_labels, test_pred)

    # Visualize
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {train_loss.item():.4f} | Test loss: {test_loss.item():.4f}")


Epoch: 0 | Train loss: 1.6627 | Test loss: 72843.2344
Epoch: 100 | Train loss: 1.1304 | Test loss: 1.6300
Epoch: 200 | Train loss: 0.8448 | Test loss: 2.3394
Epoch: 300 | Train loss: 0.7314 | Test loss: 3.3536
Epoch: 400 | Train loss: 0.6239 | Test loss: 5.8505
Epoch: 500 | Train loss: 0.5120 | Test loss: 8.8244
Epoch: 600 | Train loss: 0.3607 | Test loss: 11.6886
Epoch: 700 | Train loss: 0.2991 | Test loss: 16.3952
Epoch: 800 | Train loss: 0.2297 | Test loss: 30.6491
Epoch: 900 | Train loss: 0.1236 | Test loss: 20.9851


In [41]:
print(Y_train_labels, Y_test_labels)

tensor([1, 2, 2, 4, 3, 2, 4, 0, 3, 1, 2, 0, 1, 2, 2, 0, 1, 4, 2, 0, 0, 4, 2, 4,
        3, 0, 3, 3, 3, 3, 3, 1, 3, 4, 4, 2, 3, 1, 1, 2, 3, 0, 3, 4, 0, 4, 2, 2,
        2, 4, 1, 1, 0, 0, 2, 0, 2, 1, 0, 4, 2, 2, 0, 4, 2, 3, 4, 0, 4, 0, 1, 0,
        3, 2, 1, 2, 1, 1, 3, 1, 3, 0, 3, 3, 4, 0, 0, 4, 4, 4, 1, 1, 4, 2, 1, 4,
        0, 3, 3, 4, 0, 4, 0, 4, 0, 4, 0, 0, 4, 4, 4, 1, 2, 4, 1, 4, 2, 1, 0, 1,
        3, 1, 4, 0, 2, 1, 3, 4, 2, 0, 0, 3, 4, 2, 1, 1, 3, 0, 2, 0, 0, 3, 0, 0,
        2, 2, 4, 0, 3, 1, 2, 2, 0, 3, 4, 1, 0, 2, 1, 0, 1, 3, 4, 4, 1, 1, 3, 0,
        1, 1, 4, 2, 2, 1, 4, 3, 3, 3, 0, 3, 4, 3, 3, 0, 3, 2, 1, 2, 4, 4, 4, 0,
        1, 1, 3, 4, 2, 4, 1, 1]) tensor([3, 1, 1, 2, 1, 4, 3, 3, 1, 2, 3, 0, 3, 2, 0, 1, 3, 2, 1, 4, 2, 2, 2, 0,
        2, 0, 2, 0, 4, 2, 1, 3, 3, 2, 0, 1, 4, 2, 4, 2, 3, 3, 3, 1, 0, 4, 0, 3,
        1, 3])


# Use BCEWithLogits() Loss function

In [133]:
import torch
import torch.nn as nn

class EmotionClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=700, out_features=2048)
        self.bn1 = nn.BatchNorm1d(2048)
        self.dropout1 = nn.Dropout(0.5)

        self.layer2 = nn.Linear(2048, 4096)
        self.bn2 = nn.BatchNorm1d(4096)
        self.dropout2 = nn.Dropout(0.5)

        self.layer3 = nn.Linear(4096, 4096)
        self.bn3 = nn.BatchNorm1d(4096)
        self.dropout3 = nn.Dropout(0.5)

        self.layer4 = nn.Linear(4096, 2048)
        self.bn4 = nn.BatchNorm1d(2048)
        self.dropout4 = nn.Dropout(0.5)

        self.layer5 = nn.Linear(2048, 1024)
        self.bn5 = nn.BatchNorm1d(1024)
        self.dropout5 = nn.Dropout(0.5)

        self.layer6 = nn.Linear(1024, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.dropout6 = nn.Dropout(0.5)

        self.layer7 = nn.Linear(512, 256)
        self.bn7 = nn.BatchNorm1d(256)
        self.dropout7 = nn.Dropout(0.5)

        self.output_layer = nn.Linear(256, 5)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.relu(self.bn1(self.layer1(x)))
        #x = self.dropout1(x)

        x = torch.relu(self.bn2(self.layer2(x)))
        #x = self.dropout2(x)

        x = torch.relu(self.bn3(self.layer3(x)))
        #x = self.dropout3(x)

        x = torch.relu(self.bn4(self.layer4(x)))
        #x = self.dropout4(x)

        x = torch.relu(self.bn5(self.layer5(x)))
        #x = self.dropout5(x)

        x = torch.relu(self.bn6(self.layer6(x)))
        #x = self.dropout6(x)

        x = torch.relu(self.bn7(self.layer7(x)))
        #x = self.dropout7(x)

        x = self.output_layer(x)
        return x

model = EmotionClassification()


In [134]:
# Define loss function for multi-label classification
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.005)

# TRAINING LOOP
epochs = 500

for epoch in range(epochs):
    model.train()

    # Forward pass
    train_logits = model(X_train)

    # Calculate loss using the one-hot encoded labels
    train_loss = loss_function(train_logits, Y_train)

    # Get predictions using sigmoid
    train_pred = torch.sigmoid(train_logits)  # Use sigmoid for probabilities
    train_pred_labels = (train_pred > 0.5).float()  # Convert to one-hot format

    # Calculate accuracy
    train_acc = accuracy_fn(Y_train, train_pred_labels)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass
    train_loss.backward()

    # Update weights
    optimizer.step()

    # Evaluation
    model.eval()

    with torch.inference_mode():
        test_logits = model(X_test)

        # Calculate test loss using one-hot encoded labels
        test_loss = loss_function(test_logits, Y_test)

        # Get predictions using sigmoid
        test_pred = torch.sigmoid(test_logits)
        test_pred_labels = (test_pred > 0.5).float()  # Convert to one-hot format

        # Calculate test accuracy
        test_acc = accuracy_fn(Y_test, test_pred_labels)

    # Visualize
    if epoch % 50 == 0:
        print(f"Epoch: {epoch} | Train loss: {train_loss.item():.4f} | Test loss: {test_loss.item():.4f}")


Epoch: 0 | Train loss: 0.7362 | Test loss: 0.6957
Epoch: 50 | Train loss: 0.3475 | Test loss: 0.5994
Epoch: 100 | Train loss: 0.2399 | Test loss: 0.5509
Epoch: 150 | Train loss: 0.1838 | Test loss: 0.5364
Epoch: 200 | Train loss: 0.1492 | Test loss: 0.5347
Epoch: 250 | Train loss: 0.1258 | Test loss: 0.5389


KeyboardInterrupt: 

# Model evaluation

In [135]:
model.eval()

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    test_logits = model(X_test)
    test_probabilities = torch.sigmoid(test_logits)
    test_pred_labels = torch.argmax((test_probabilities), dim = 1)

    Y_test_single = torch.argmax(Y_test, dim = 1)


    for i in range(0,50):
      if(test_pred_labels[i] == Y_test_single[i]):
        correct_predictions += 1
      total_predictions +=1

# Calculate accuracy
accuracy = (correct_predictions / total_predictions)*100

print(f'Accuracy of the model on the test set: {accuracy:.4f}%')

print(f'True label: {Y_test_single}')
print(f'Predicted label: {test_pred_labels}')

Accuracy of the model on the test set: 26.0000%
True label: tensor([3, 1, 1, 2, 1, 4, 3, 3, 1, 2, 3, 0, 3, 2, 0, 1, 3, 2, 1, 4, 2, 2, 2, 0,
        2, 0, 2, 0, 4, 2, 1, 3, 3, 2, 0, 1, 4, 2, 4, 2, 3, 3, 3, 1, 0, 4, 0, 3,
        1, 3])
Predicted label: tensor([4, 1, 1, 1, 1, 4, 0, 4, 4, 4, 1, 0, 4, 4, 1, 4, 0, 4, 2, 0, 4, 2, 0, 0,
        4, 4, 0, 1, 4, 0, 2, 2, 1, 0, 4, 2, 4, 2, 4, 0, 1, 0, 1, 1, 4, 1, 0, 1,
        0, 1])


#BACK TO CROSSENTROPY

In [130]:
# Define loss function for multi-class classification
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

# TRAINING LOOP
epochs = 500

for epoch in range(epochs):
    model.train()

    # Forward pass
    train_logits = model(X_train)

    # Convert one-hot encoded labels to class indices
    Y_train_indices = Y_train.argmax(dim=1)  # Get the class indices

    # Calculate loss using the class indices
    train_loss = loss_function(train_logits, Y_train_indices)

    # Get predictions using softmax for probabilities
    train_pred_probs = torch.softmax(train_logits, dim=1)
    train_pred_labels = torch.argmax(train_pred_probs, dim=1)  # Get predicted class labels

    # Calculate accuracy
    train_acc = accuracy_fn(Y_train_indices, train_pred_labels)

    # Zero the gradients
    optimizer.zero_grad()

    # Backward pass
    train_loss.backward()

    # Update weights
    optimizer.step()

    # Evaluation
    model.eval()

    with torch.inference_mode():
        test_logits = model(X_test)

        # Calculate test loss using class indices
        Y_test_indices = Y_test.argmax(dim=1)  # Get the class indices for the test set
        test_loss = loss_function(test_logits, Y_test_indices)

        # Get predictions using softmax
        test_pred_probs = torch.softmax(test_logits, dim=1)
        test_pred_labels = torch.argmax(test_pred_probs, dim=1)  # Get predicted class labels

        # Calculate test accuracy
        test_acc = accuracy_fn(Y_test_indices, test_pred_labels)

    # Visualize
    if epoch % 50 == 0:
        print(f"Epoch: {epoch} | Train loss: {train_loss.item():.4f} | Test loss: {test_loss.item():.4f}")


Epoch: 0 | Train loss: 1.6692 | Test loss: 1.6130
Epoch: 50 | Train loss: 1.3710 | Test loss: 1.7269
Epoch: 100 | Train loss: 0.3103 | Test loss: 2.7690


KeyboardInterrupt: 